In [32]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyrootutils
from IPython.display import display

pyrootutils.set_root(os.path.abspath(".."), pythonpath=True)
    
from src.utils.data import load_news, load_behaviors
from src.utils.hydra import RunCollection

DATA_DIR = "../data"


In [33]:
def get_metrics(runs, split="dev", best_on="AUC", index_by=None, **kwargs):
    dev_metrics = pd.concat([pd.read_csv(run.get_file_path("metrics_dev.csv")).assign(run=run) for run in runs])
    dev_metrics = dev_metrics.reset_index(drop=True)
    best_epoch = dev_metrics.groupby("run")[best_on].idxmax()
    
    metrics = pd.concat([pd.read_csv(run.get_file_path(f"metrics_{split}.csv")).assign(run=run) for run in runs])
    metrics = metrics.reset_index(drop=True)
    best_metrics = metrics.loc[best_epoch]
    for key, value in kwargs.items():
        best_metrics[key] = best_metrics["run"].apply(value)
    if index_by:
        return best_metrics.set_index(index_by, drop=True).sort_index()
    return best_metrics

def get_metrics_per_epoch(runs, split="dev", index_by=None, **kwargs):
    metrics = pd.concat([pd.read_csv(run.get_file_path(f"metrics_{split}.csv")).assign(run=run) for run in runs])
    metrics = metrics.reset_index(drop=True)
    for key, value in kwargs.items():
        metrics[key] = metrics["run"].apply(value)
    if index_by:
        return metrics.set_index(index_by, drop=True).sort_index()
    return metrics

In [34]:
hparam_runs = RunCollection.from_path("../multirun")\
    .filter_by_job("train_recommender")\
    .filter(lambda run: "hparams" in run.config.tags)\
    .filter(lambda run: "metrics_dev.csv" in run.list_files()) # only completed runs


In [35]:
baseline_runs = hparam_runs.filter_by_override("+experiment", "baseline")
baseline_metrics = get_metrics(
    baseline_runs,
    index_by=["model", "history_mask"],
    model=lambda run: run.overrides["+model"],
    history_mask=lambda run: run.config.use_history_mask,
)
display(baseline_metrics)
display(baseline_metrics.loc[baseline_metrics.groupby("model")["AUC"].idxmax()])

AUC       MRR    NDCG@5   NDCG@10  epoch  \
model history_mask                                                  
naml  False         0.684824  0.329870  0.365823  0.428846      4   
      True          0.677233  0.320281  0.356732  0.420236      5   
nrms  False         0.658464  0.309063  0.342386  0.407371      5   
      True          0.678373  0.327599  0.361544  0.424687      5   
tanr  False         0.669750  0.318366  0.350656  0.414704      5   
      True          0.666312  0.315725  0.348641  0.411907      5   

                                                            run  
model history_mask                                               
naml  False         Run(path=../multirun/2023-06-12/16-21-31/0)  
      True          Run(path=../multirun/2023-06-12/16-21-31/1)  
nrms  False         Run(path=../multirun/2023-06-12/14-44-55/0)  
      True          Run(path=../multirun/2023-06-12/14-44-55/1)  
tanr  False         Run(path=../multirun/2023-06-12/14-44-55/2)  
      True          Run(path=../multirun/2023-06-12/14-44-55/3)

,,AUC,MRR,NDCG@5,NDCG@10,epoch,run
model,history_mask,,,,,,
naml,False,0.684824,0.329870,0.365823,0.428846,4,Run(path=../multirun/2023-06-12/16-21-31/0)
nrms,True,0.678373,0.327599,0.361544,0.424687,5,Run(path=../multirun/2023-06-12/14-44-55/1)
tanr,False,0.669750,0.318366,0.350656,0.414704,5,Run(path=../multirun/2023-06-12/14-44-55/2)


In [36]:
hierec_runs = hparam_runs.filter(lambda run: "hierec" in run.overrides["+experiment"])
hierec_metrics = get_metrics(
    hierec_runs,
    index_by=["model", "history_mask"],
    model=lambda run: run.overrides["+experiment"].split("_")[0],
    history_mask=lambda run: run.config.use_history_mask,
)
display(hierec_metrics)
display(hierec_metrics.loc[hierec_metrics.groupby("model")["AUC"].idxmax()])

AUC       MRR    NDCG@5   NDCG@10  epoch  \
model history_mask                                                  
naml  False         0.662146  0.312854  0.345113  0.410379      2   
      True          0.674250  0.319615  0.353117  0.417583      5   
nrms  False         0.652659  0.311511  0.342616  0.407263      4   
      True          0.676315  0.328161  0.362376  0.425634      5   
tanr  False         0.658235  0.314197  0.346950  0.411404      4   
      True          0.667404  0.318588  0.350952  0.415962      5   

                                                            run  
model history_mask                                               
naml  False         Run(path=../multirun/2023-06-12/16-03-44/0)  
      True          Run(path=../multirun/2023-06-12/16-03-44/1)  
nrms  False         Run(path=../multirun/2023-06-12/14-57-07/0)  
      True          Run(path=../multirun/2023-06-12/14-57-07/1)  
tanr  False         Run(path=../multirun/2023-06-12/14-57-07/2)  
      True          Run(path=../multirun/2023-06-12/14-57-07/3)

,,AUC,MRR,NDCG@5,NDCG@10,epoch,run
model,history_mask,,,,,,
naml,True,0.674250,0.319615,0.353117,0.417583,5,Run(path=../multirun/2023-06-12/16-03-44/1)
nrms,True,0.676315,0.328161,0.362376,0.425634,5,Run(path=../multirun/2023-06-12/14-57-07/1)
tanr,True,0.667404,0.318588,0.350952,0.415962,5,Run(path=../multirun/2023-06-12/14-57-07/3)


In [37]:
multi_interest_runs = hparam_runs.filter(lambda run: "multi_interest" in run.overrides["+experiment"])
multi_interest_metrics = get_metrics(
    multi_interest_runs,
    index_by=["model", "history_mask", "n_interest_vectors"],
    model=lambda run: run.overrides["+experiment"].split("_")[0],
    history_mask=lambda run: run.config.use_history_mask,
    n_interest_vectors=lambda run: run.config.model.user_encoder.n_interest_vectors
)
display(multi_interest_metrics)
multi_interest_metrics = multi_interest_metrics.reset_index(level="n_interest_vectors")
multi_interest_metrics = multi_interest_metrics[multi_interest_metrics["n_interest_vectors"] != 1]
multi_interest_metrics = multi_interest_metrics.set_index("n_interest_vectors", append=True)
display(multi_interest_metrics.loc[multi_interest_metrics.groupby("model")["AUC"].idxmax()])

AUC       MRR    NDCG@5   NDCG@10  \
model history_mask n_interest_vectors                                           
naml  False        1                   0.684764  0.329485  0.365053  0.428359   
                   4                   0.627116  0.287156  0.313630  0.376650   
                   16                  0.650986  0.300137  0.329913  0.394253   
                   32                  0.668336  0.310847  0.340232  0.405489   
                   48                  0.673778  0.320066  0.352016  0.415405   
                   64                  0.674898  0.319567  0.351810  0.416314   
      True         1                   0.676562  0.322725  0.358234  0.420974   
                   4                   0.630421  0.299526  0.325673  0.387773   
                   16                  0.653178  0.309235  0.338908  0.401661   
                   32                  0.655951  0.316177  0.345929  0.409312   
                   48                  0.653655  0.316206  0.342796  0.406568   
                   64                  0.649045  0.306836  0.335438  0.399364   
nrms  False        1                   0.674714  0.326189  0.360289  0.422647   
                   4                   0.663039  0.317804  0.349623  0.412672   
                   16                  0.669293  0.321003  0.354082  0.417580   
                   32                  0.665438  0.321534  0.353431  0.416300   
                   48                  0.667705  0.322560  0.355941  0.418189   
                   64                  0.667612  0.319022  0.352168  0.415960   
      True         1                   0.672980  0.325206  0.358990  0.422011   
                   4                   0.631043  0.291027  0.317148  0.382655   
                   16                  0.655244  0.312263  0.341382  0.405492   
                   32                  0.658419  0.313590  0.345064  0.408068   
                   48                  0.657476  0.314931  0.345322  0.408651   
                   64                  0.659481  0.315505  0.345776  0.408737   
tanr  False        1                   0.666914  0.316901  0.349156  0.413094   
                   4                   0.642722  0.298200  0.326381  0.391268   
                   16                  0.645228  0.299607  0.325861  0.392107   
                   32                  0.645966  0.305202  0.331155  0.398012   
                   64                  0.643644  0.299705  0.327018  0.393269   
      True         1                   0.668060  0.316984  0.349942  0.413294   
                   4                   0.633513  0.301248  0.329244  0.391670   
                   32                  0.646742  0.306626  0.336629  0.398399   
                   48                  0.648621  0.307258  0.336956  0.400673   
                   64                  0.650847  0.309249  0.337800  0.401798   

                                       epoch  \
model history_mask n_interest_vectors          
naml  False        1                       4   
                   4                       1   
                   16                      3   
                   32                      2   
                   48                      2   
                   64                      4   
      True         1                       3   
                   4                       5   
                   16                      5   
                   32                      5   
                   48                      5   
                   64                      5   
nrms  False        1                       5   
                   4                       5   
                   16                      5   
                   32                      5   
                   48                      5   
                   64                      5   
      True         1                       5   
                   4                       4   
                   16                      5   
                   32  

,,,AUC,MRR,NDCG@5,NDCG@10,epoch,run
model,history_mask,n_interest_vectors,,,,,,
naml,False,64,0.674898,0.319567,0.351810,0.416314,4,Run(path=../multirun/2023-06-12/14-56-49/10)
nrms,False,16,0.669293,0.321003,0.354082,0.417580,5,Run(path=../multirun/2023-06-12/14-46-08/4)
tanr,True,64,0.650847,0.309249,0.337800,0.401798,5,Run(path=../multirun/2023-06-12/14-47-09/11)
